In [6]:
import requests
import json
from bs4 import BeautifulSoup
import os
import re
from DrissionPage import ChromiumPage, ChromiumOptions
import time
import threading
from urllib.parse import urlparse

In [2]:
SERPER_API_KEY = "c1260f4908aaa9209616a9e7348fffdbf08a44dd"  
SEARCH_QUERY = "红岩网校"                        
RESULT_COUNT = 5                  
headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }
    
    # 构建请求数据
data = {
        "q": SEARCH_QUERY,
        "num": RESULT_COUNT
    }

In [3]:
# 定义搜索函数
def simple_serper_search():
    try:
        
        response = requests.post( "https://google.serper.dev/search", headers=headers, data=json.dumps(data))
        response.raise_for_status()  # 检查请求是否成功
        
    # 解析返回的JSON数据
        response_result = response.json()
    #查看是否有answerBox和knowledgeGraph
        results ={}
        box_result = []
        knowledge_result = []
        
        if 'answerBox' in response_result:
            print("存在 answerBox:")
            print(response_result['answerBox'])
            box_result.append(response_result['answerBox'])
            results["answerBox"] = box_result
        if 'knowledgeGraph' in response_result:
            print("存在 knowledgeGraph:")
            print(response_result['knowledgeGraph'])
            knowledge_result.append(response_result['knowledgeGraph'])
            results["knowledgeGraph"] = knowledge_result
    #解析organic结果
        result = []    
        for i, item in enumerate(response_result['organic'], 1):
            
            """print(f"结果 {i}:")
            print("*" * 100)
            
            print(f"  链接: {item.get('link', '无链接')}")            
            print("*" * 100)"""
            result.append(item.get('title', '无标题'))
            result.append(item.get('snippet', '无摘要'))
            result.append(item.get('link', '无链接'))
            results[f"result{i}"] = result
            result = []  # 重置result列表以准备下一次循环
        return results
        
    except requests.exceptions.RequestException as e:
        print(f"网络请求出错: {e}")
    except json.JSONDecodeError as e:
        print(f"解析JSON响应出错: {e}")
    except  Exception as e:
        print(f"发生未知错误: {e}")

In [ ]:
"""# 定义全展示搜索函数
def all_serper_search():
    try:
        
        response = requests.post( "https://google.serper.dev/search", headers=headers, data=json.dumps(data))
        response.raise_for_status()  # 检查请求是否成功
        
    # 解析返回的JSON数据
        result = response.json()
        return result
        
    except requests.exceptions.RequestException as e:
        print(f"网络请求出错: {e}")
    except json.JSONDecodeError as e:
        print(f"解析JSON响应出错: {e}")
    except  Exception as e:
        print(f"发生未知错误: {e}")
"""

In [ ]:
"""#查看全部结果
result = all_serper_search()
# Print 完整JSON内容
print("========== 搜索结果JSON内容 ==========\n")
print(json.dumps(result, indent=2, ensure_ascii=False))
# 将结果保存为JSON文件
with open('serper_search_result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=2, ensure_ascii=False)
print("JSON已保存到 serper_search_result.json")
"""

In [4]:
#实际实施
results = simple_serper_search()
#print("搜索结果:", result)

  摘要: 红岩网校工作站是学校唯一一个从事互联网产品开发运营的校级学生组织，在校团委的指导下，走出了一大批优秀的IT互联网人才，深受腾讯、 阿里巴巴、字节跳动、华为等知名 ...
  摘要: 红岩网校是由重庆邮电大学党委于2000年牵头创建、学生自主运营管理的网络思想政治教育平台，以红岩精神为核心构建思想政治工作阵地。其采用“一校五网十三站”架构，涵盖思政 ...
  摘要: 红岩网校- 关于我们. Contribute to RedrockTeam/about-us development by creating an account on GitHub.
  摘要: 重庆邮电大学“红岩网校”成立十年来，充分发挥专业学科特色和优势，主动适应信息社会和网络文化的迅猛发展，实施“六个一”工程，开创了大学生网络思想政治教育工作新局面， ...
  摘要: “'红岩网校'是以'红岩精神'为核心的大学生思想政治教育网上阵地。”重庆邮电学院党委书记徐仲伟这样定义“红岩网校”。 现任“红岩网校”工作站站长的李云宏说，“红岩网校 ...
搜索结果: ['红岩网校工作站是学校唯一一个从事互联网产品开发运营的校级学生组织，在校团委的指导下，走出了一大批优秀的IT互联网人才，深受腾讯、 阿里巴巴、字节跳动、华为等知名 ...', 'https://redrock.team/', '红岩网校是由重庆邮电大学党委于2000年牵头创建、学生自主运营管理的网络思想政治教育平台，以红岩精神为核心构建思想政治工作阵地。其采用“一校五网十三站”架构，涵盖思政 ...', 'https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299', '红岩网校- 关于我们. Contribute to RedrockTeam/about-us development by creating an account on GitHub.', 'https://github.com/RedrockTeam/about-us', '重庆邮电大学“红岩网校”成立十年来，充分发挥专业学科特色和优势，主动适应信息社会和网络文化的迅猛发展，实施“六个一”工程，开创了大学生网络思想政治教育工作新局面， ...', 'http://www.mo

In [ ]:
"""#用现有本地json文件测试解析代码
import glob
json_files = glob.glob("*.json")
with open(json_files[0], 'r', encoding='utf-8') as f:
    response_result = json.load(f)
#查看是否有answerBox和knowledgeGraph
results = {}
box_result = []
knowledge_result = []
        
if 'answerBox' in response_result:
    print("存在 answerBox:")
    print(response_result['answerBox'])
    box_result.append(response_result['answerBox'])
    results["answerBox"] = box_result
if 'knowledgeGraph' in response_result:
    print("存在 knowledgeGraph:")
    print(response_result['knowledgeGraph'])
    knowledge_result.append(response_result['knowledgeGraph'])
    results["knowledgeGraph"] = knowledge_result
#解析organic结果
result = []    
for i, item in enumerate(response_result['organic'], 1):
            result.append(item.get('title', '无标题'))
            result.append(item.get('snippet', '无摘要'))
            result.append(item.get('link', '无链接'))
            results[f"result{i}"] = result
            result = []  # 重置result列表以准备下一次循环
print(results)"""

{'result1': ['红岩网校工作站', '红岩网校工作站是学校唯一一个从事互联网产品开发运营的校级学生组织，在校团委的指导下，走出了一大批优秀的IT互联网人才，深受腾讯、 阿里巴巴、字节跳动、华为等知名 ...', 'https://redrock.team/'], 'result2': ['红岩网校', '红岩网校是由重庆邮电大学党委于2000年牵头创建、学生自主运营管理的网络思想政治教育平台，以红岩精神为核心构建思想政治工作阵地。其采用“一校五网十三站”架构， ...', 'https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299'], 'result3': ['RedrockTeam/about-us: 红岩网校- 关于我们', 'RedrockTeam/about-us · Folders and files · Latest commit · History · Repository files navigation. README. About us - Redrock. About. 红岩网校- 关于我们 ...', 'https://github.com/RedrockTeam/about-us'], 'result4': ['重庆邮电大学“红岩网校”实施“六个一”工程开创大学生网络 ...', '目前，“红岩网校”已发展成为“一校、五网、十三站、229个一级栏目、834个二级栏目、累计注册用户4万余人、日均点击量突破10万人次”的素质教育网络大平台，满足了学生多样化的 ...', 'http://www.moe.gov.cn/s78/A12/gongzuo/moe_2154/201101/t20110105_114187.html'], 'result5': ['掌上重邮 - 红岩网校工作站', '学生查询. 输入姓名/学号即可查询学生信息 ; 考试/成绩查询. 考试安排，考场信息，考试成绩一键查看 ; 备忘添加. 直接添加至课表，更便捷的使用感受 ; 课前提醒. 每周推送/每日 ...', 'https://m.app.redrock.team/']}


In [16]:
#匹配关键词排序
def match_select_func(results,words):
    match_count = {}
    # 遍历 results 中的各个结果，计算关键词匹配次数
    for result_key in ['result1', 'result2', 'result3']:
        #将result中的前两个元素合并为一个字符串,匹配标题和摘要
        if result_key in results:
            result_list = results[result_key]
            text = result_list[0] + result_list[1]
            count = 0
            # 统计关键词匹配次数
            for word in words:
                if word in text:
                    count += 1
            match_count[result_key] = count
    
    # 按匹配次数排序（降序）
    sorted_results = sorted(match_count.items(), key=lambda x: x[1], reverse=True)
    
    # 提取前三个结果的链接（第三个元素）
    links = []
    for i in range(min(3, len(sorted_results))):
        result_key = sorted_results[i][0]
        link = results[result_key][2]
        links.append(link)
    
    return links
"""words = ["红岩网校", "重庆邮电", "学习平台"]
top_links = match_select_func(results, words)
print("匹配结果前三的链接：", top_links)"""


'words = ["红岩网校", "重庆邮电", "学习平台"]\ntop_links = match_select_func(results, words)\nprint("匹配结果前三的链接：", top_links)'

In [17]:
##匹配关键词排序
"""注意：这里的words是LLM根据用户输入与搜索结果生成的关键词"""
words = ["红岩网校", "重庆邮电", "学习平台"]
selected_links = match_select_func(results, words)
print("匹配结果前三的链接：", selected_links)

匹配结果前三的链接： ['https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299', 'https://redrock.team/', 'https://github.com/RedrockTeam/about-us']


In [ ]:
"""
#爬虫设置
def craw(url):
    r = requests.get(url)
    #print(f"爬取链接: {url} 状态码: {r.status_code}")
    #print(f"爬取内容前500字符: {r.text[:500]}")
    return r.text

def parse(html_content):

    finder = BeautifulSoup(html_content, 'html.parser')
    
    # 找到所有文本内容的span标签
    text_span1 = finder.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    text_span2 = finder.find_all('p')
    text_span3 = finder.find_all('span')
    text_span4 = finder.find_all('div')
    text_span5 = finder.find_all('li')
    text_spans = text_span1 + text_span2 + text_span3 + text_span4 + text_span5
    
    # 提取文本
    contents = [span.get_text() for span in text_spans]
    print("提取的文本内容:", contents)
    
    return contents

def filter_chinese(contents):
    """"""只保留中文字符""""""
    import re
    filtered_contents = []
    for content in contents:
        # 提取中文字符
        chinese_only = re.sub(r'[^\u4e00-\u9fff]', '', content)
        if chinese_only:  # 如果有中文内容才保存
            filtered_contents.append(chinese_only)
    #排序，选择前三个最短的
    #filtered_contents = sorted(filtered_contents, key=len)[:3]
    return filtered_contents"""

In [ ]:
"""#多线程爬虫
def super_craw(urls, crawed_processed):
    html = craw(urls)
    content = parse(html)
    filtered_content = filter_chinese(content) 
    crawed_processed.append({"content": filtered_content})


    """

In [ ]:
"""selected_links = ['https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299', 'https://redrock.team/', 'https://github.com/RedrockTeam/about-us']
crawed_processed = []
lock = threading.Lock()
threads = []
with lock:
#设置多线程并开始爬取
    for i in range(3):
        threads.append(threading.Thread(target=super_craw, args=(selected_links[i], crawed_processed)))

    for t in threads:
        t.start()
    for t in threads:
        t.join()
print(crawed_processed)"""

提取的文本内容: ['']
提取的文本内容: []
提取的文本内容: ['Navigation Menu', 'Search code, repositories, users, issues, pull requests...', '\n        Provide feedback\n      ', '\n        Saved searches\n      ', 'Use saved searches to filter your results more quickly', 'RedrockTeam/about-us', 'Folders and files', 'Latest commit', 'History', 'Repository files navigation', 'About us - Redrock', 'About', 'Resources', '        Uh oh!\n', 'Stars', 'Watchers', 'Forks', '\nReleases', '\nPackages\n      0', '        Uh oh!\n', '\nContributors\n      4', '        Uh oh!\n', 'Languages', 'Footer', 'Footer navigation', 'We read every piece of feedback, and take your input very seriously.', '\n            To see all available qualifiers, see our documentation.\n          ', '\n        红岩网校 - 关于我们\n      ', '\n        红岩网校 - 关于我们\n      ', ' There was an error while loading. Please reload this page.\n', 'There was an error while loading. Please reload this page.', ' There was an error while loading. Please reload this 

In [ ]:
"""#验证百度反爬爬虫
http_content = craw("https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299")
print(http_content)"""

<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><meta http-equiv="X-UA-Compatible"content="IE=edge"><meta name="viewport"content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"><title>百度安全验证</title><style>body{margin:0;padding:0}</style></head><body></body><!--loadScript--><script>function ieOnEnd(script,cb){script.onreadystatechange=function(){if(this.readyState!=='complete'&&this.readyState!=='loaded'){return}this.onreadystatechange=null;cb(null,script)}}function stdOnEnd(script,cb){script.onload=function(){this.onerror=this.onload=null;cb(null,script)};script.onerror=function(e){this.onerror=this.onload=null;cb(new Error(`Failed to load ${this.src}`),script)}}function loadScript(src,opts,cb){var script=document.createElement('script');if(typeof opts==='function'){cb=opts;opts={}}opts=opts||{};cb=cb||function(){};script.type=opts.type||'text/javascript';script.charset=opts.charset||'utf8';script.async='async'in opts?!!opts.async:true;script.src

In [ ]:
"""原装强力爬虫
import os
import time
from bs4 import BeautifulSoup

try:
    from DrissionPage import ChromiumPage, ChromiumOptions
except ImportError:
    ChromiumPage = None
    ChromiumOptions = None
    print("未安装 DrissionPage，请先运行: !pip install drissionpage")


def super_craw_baidu(url=None, keyword=None, user_data_dir=None, wait_sec=2):
    """"""使用 DrissionPage(有头) 抓取百度百科，尽量绕过安全验证，并做结构化解析。

    参数:
      - url / keyword: 目标（url 优先）
      - user_data_dir: 持久化目录（默认 ./drission_profile_baidu）
      - wait_sec: 页面加载后额外等待秒数

    返回:
      dict: {title, summary, info_box, url}，失败返回 None
    """"""
    if not url:
        if keyword:
            url = f"https://baike.baidu.com/item/{keyword}"
        else:
            print("错误: 必须提供 url 或 keyword")
            return None

    if ChromiumPage is None:
        return None

    if user_data_dir is None:
        user_data_dir = os.path.abspath("drission_profile_baidu")

    # 有头 + 持久化用户数据目录
    co = ChromiumOptions()
    co.set_paths(user_data_path=user_data_dir)
    # 不要无头
    # co.headless(True)  # 不用
    co.set_argument("--disable-blink-features=AutomationControlled")
    co.set_argument("--no-first-run")
    co.set_argument("--no-default-browser-check")

    page = ChromiumPage(co)
    try:
        print(f"正在访问: {url}")
        page.get(url)

        time.sleep(wait_sec)

        html = page.html or ""
        print(html[:2000])
        if "百度安全验证" in html:
            print("⚠️ 触发百度安全验证：请在弹出的浏览器窗口中手动完成验证。")
            print(f"提示：验证通过后会写入持久化目录：{user_data_dir}")
            print("完成验证后，请回到这里再次运行 Cell 14（会复用同一份 profile）。")
            return None

        soup = BeautifulSoup(html, "html.parser")

        # 标题
        h1 = soup.find("h1")
        title = h1.get_text(" ", strip=True) if h1 else ""

        # 摘要
        summary_div = soup.find("div", class_="lemma-summary")
        summary = summary_div.get_text(" ", strip=True) if summary_div else ""

        # 基本信息栏
        info_box = {}
        basic_info = soup.find("div", class_="basic-info")
        if basic_info:
            dts = basic_info.find_all("dt", class_="basicInfo-item")
            dds = basic_info.find_all("dd", class_="basicInfo-item")
            for dt, dd in zip(dts, dds):
                k = dt.get_text(" ", strip=True).replace("\xa0", "")
                v = dd.get_text(" ", strip=True)
                if k:
                    info_box[k] = v

        print(f"✅ 爬取成功: {title or url}")
        return {"title": title, "summary": summary, "info_box": info_box, "url": url}

    except Exception as e:
        print(f"❌ 运行失败: {e}")
        return None

    finally:
        try:
            page.quit()
        except Exception:
            pass
"""

In [ ]:
"""原装强力爬虫演示运行
import inspect
import json

# 自检：确认你现在跑到的是 DrissionPage 版本，不会再出现 requests 的 403/移动UA那套输出
src = inspect.getsource(super_craw_baidu)
if "DrissionPage" in src or "ChromiumPage" in src:
    print("✅ 检测到正确的 DrissionPage 版本函数")
else:
    print("❌ 当前不是 DrissionPage 版本，请先覆盖并运行 Cell 13")
    raise RuntimeError("函数版本不对")

info = super_craw_baidu(url="https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299")

if info:
    print("\n--- 抓取到的结构化结果 ---")
    print(json.dumps(info, indent=2, ensure_ascii=False))
else:
    print("\n未获取到内容：如果弹出了安全验证，请在窗口中完成验证后再重新运行本单元格。")
    """

✅ 检测到正确的 DrissionPage 版本函数
正在访问: https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299
<!DOCTYPE html><html class=""><head><meta charset="UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="referrer" content="always"><title>红岩网校_百度百科</title><meta name="description" content="红岩网校是由重庆邮电大学党委于2000年牵头创建、学生自主运营管理的网络思想政治教育平台，以红岩精神为核心构建思想政治工作阵地。其采用“一校五网十三站”架构，涵盖思政宣传、资源服务、校园社交等功能，开发了四六级查询系统、掌上重邮APP、开源镜像站等实用工具，日均访问量达10万人次。该平台初期仅设3个一级栏目，经历12年发展完成11次改版，栏目扩展至229个，日访问量从1500人次增至10万余人次。技术架构采用Linux+Apache+MySQL+PHP组合，建立导师制培训机制，形成校党委领导、职能部门指导、学生团队运维的三级管理体系。工作站成员来自全校各学院，通过部门协作完成开发维护任务，累计培养的IT人才受到腾讯、华为等企业青睐。"><meta name="keywords" content="红岩网校, 红岩网校主要网站, 红岩网校网站发展, 红岩网校核心成员"><meta itemprop="dateUpdate" content="2025-09-22 13:40:23"><link rel="alternate" hreflang="x-default" href="https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299"><link rel="alternate" hreflang="zh" href="https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/110329

In [1]:
def craw1(url):
    """requests 抓取（快，适合大多数非百度站点）"""
    try:
        r = requests.get(url, timeout=10)
        return r.text
    except Exception as e:
        print(f"requests 抓取失败: {url} - {e}")
        return ""


def craw2(url, page):
    """浏览器抓取（速度优先）：只负责返回 html，不做解析"""
    try:
        page.get(url)
        page.wait.doc_loaded()
        return page.html or ""
    except Exception as e:
        print(f"浏览器抓取失败: {url} - {e}")
        return ""

def parse1(html_content):
    finder = BeautifulSoup(html_content, 'html.parser')

    text_span1 = finder.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    text_span2 = finder.find_all('p')
    text_span3 = finder.find_all('span')
    text_span4 = finder.find_all('div')
    text_span5 = finder.find_all('li')
    text_spans = text_span1 + text_span2 + text_span3 + text_span4 + text_span5

    contents = [span.get_text(" ", strip=True) for span in text_spans]
    return contents

def parse2(html):
    """百度轻量解析：只取 title/description/keywords，并返回 list[str] 方便走 filter_chinese"""
    if not html:
        return [], False

    if "百度安全验证" in html:
        return [], True

    title = ""
    m = re.search(r"<title>(.*?)</title>", html, re.I | re.S)
    if m:
        title = re.sub(r"\s+", " ", m.group(1)).strip()

    desc = ""
    m = re.search(r'<meta[^>]+name=["\']description["\'][^>]+content=["\'](.*?)["\']', html, re.I | re.S)
    if m:
        desc = re.sub(r"\s+", " ", m.group(1)).strip()

    keywords = ""
    m = re.search(r'<meta[^>]+name=["\']keywords["\'][^>]+content=["\'](.*?)["\']', html, re.I | re.S)
    if m:
        keywords = re.sub(r"\s+", " ", m.group(1)).strip()

    parts = []
    if title:
        parts.append(title)
    if keywords:
        parts.append(keywords)
    if desc:
        parts.append(desc)

    return parts, False
    
def filter_chinese(contents):
    filtered_contents = []
    for content in contents:
        chinese_only = re.sub(r'[^\u4e00-\u9fff]', '', content)
        if chinese_only and len(chinese_only) >= 6:
            filtered_contents.append(chinese_only)

    # 去重（保留顺序）
    seen = set()
    dedup = []
    for x in filtered_contents:
        if x not in seen:
            seen.add(x)
            dedup.append(x)
    return dedup



In [3]:
def super_threading_for_txt_craw(urls, crawed_processed, use_threading=True, max_threads=8):
    def worker(u):
        html = craw1(u)
        # 调用 parse1（原版全量解析）
        content = parse1(html)
        filtered = filter_chinese(content)
        crawed_processed.append({"url": u, "content": filtered, "blocked": False})

    if not use_threading:
        for u in urls:
            worker(u)
        return

    threads = []
    for u in urls:
        t = threading.Thread(target=worker, args=(u,))
        threads.append(t)
        t.start()

        if len(threads) >= max_threads:
            for tt in threads:
                tt.join()
            threads = []

    for tt in threads:
        tt.join()

def chrome_super_for_txt_craw(urls, crawed_processed):
    _co = ChromiumOptions()
    _co.set_paths(user_data_path=os.path.abspath("drission_profile_baidu"))
    _co.headless(True)
    _co.no_imgs(True)

    page = ChromiumPage(_co)
    try:
        for u in urls:
            # 1. 抓取 HTML
            html = craw2(u, page)
            
            # 2. 调用 parse2（百度轻量解析）
            parts, blocked = parse2(html)

            if blocked:
                crawed_processed.append({"url": u, "content": [], "blocked": True})
                continue

            # 3. 过滤中文并保存
            filtered = filter_chinese(parts)
            crawed_processed.append({"url": u, "content": filtered, "blocked": False})
    finally:
        try:
            page.quit()
        except Exception:
            pass

In [4]:
def judge_super_craw(urls, crawed_processed, use_threading_for_requests=True):
    baidu_urls = []
    normal_urls = []

    for u in urls:
        host = (urlparse(u).netloc or "").lower()
        if "baike.baidu.com" in host or host.endswith(".baidu.com") or host == "baidu.com":
            baidu_urls.append(u)
        else:
            normal_urls.append(u)

    # 先抓普通站：requests + 多线程（快）
    if normal_urls:
        super_threading_for_txt_craw(
            normal_urls,
            crawed_processed,
            use_threading=use_threading_for_requests,
            max_threads=8
        )

    # 再抓百度：浏览器（稳）
    if baidu_urls:
        chrome_super_for_txt_craw(baidu_urls, crawed_processed)

In [7]:
selected_links = [
    'https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299',
    'https://redrock.team/',
    'https://github.com/RedrockTeam/about-us'
]

crawed_processed = []
judge_super_craw(selected_links, crawed_processed, use_threading_for_requests=True)

print(crawed_processed)

[{'url': 'https://redrock.team/', 'content': [], 'blocked': False}, {'url': 'https://github.com/RedrockTeam/about-us', 'content': ['红岩网校关于我们', '红岩网校关于我们红岩网校关于我们'], 'blocked': False}, {'url': 'https://baike.baidu.com/item/%E7%BA%A2%E5%B2%A9%E7%BD%91%E6%A0%A1/1103299', 'content': ['红岩网校百度百科', '红岩网校红岩网校主要网站红岩网校网站发展红岩网校核心成员', '红岩网校是由重庆邮电大学党委于年牵头创建学生自主运营管理的网络思想政治教育平台以红岩精神为核心构建思想政治工作阵地其采用一校五网十三站架构涵盖思政宣传资源服务校园社交等功能开发了四六级查询系统掌上重邮开源镜像站等实用工具日均访问量达万人次该平台初期仅设个一级栏目经历年发展完成次改版栏目扩展至个日访问量从人次增至万余人次技术架构采用组合建立导师制培训机制形成校党委领导职能部门指导学生团队运维的三级管理体系工作站成员来自全校各学院通过部门协作完成开发维护任务累计培养的人才受到腾讯华为等企业青睐'], 'blocked': False}]
